<a href="https://colab.research.google.com/github/Vidharshan/cip/blob/main/100_CIP_IMPLEMENTATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!git clone https://github.com/deepmind/ithaca

Cloning into 'ithaca'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 73 (delta 19), reused 68 (delta 14), pack-reused 0
Unpacking objects: 100% (73/73), 2.44 MiB | 6.22 MiB/s, done.


In [26]:
import os
os.chdir('ithaca')
!pip install --editable .[train]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/ithaca/train/ithaca
  Preparing metadata (setup.py) ... done
  Attempting uninstall: ithaca
    Found existing installation: ithaca 0.1.0
    Uninstalling ithaca-0.1.0:
      Successfully uninstalled ithaca-0.1.0
  Running setup.py develop for ithaca


In [27]:
os.chdir('train')

In [28]:
!./launch_local.sh

2023-05-18 06:11:26.899947: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-18 06:11:28.621849: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/ithaca/train/ithaca/train/experiment.py", line 29, in <module>
    from ithaca.util.loss import categorical_kl_divergence
  File "/content/ithaca/train/ithaca/ithaca/util/loss.py", line 16, in <module>
    from flax.deprecated import nn
ModuleNotFoundError: No module named 'flax.deprecated'


In [29]:
!curl --output checkpoint.pkl https://storage.googleapis.com/ithaca-resources/models/checkpoint_v1.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  192M  100  192M    0     0   133M      0  0:00:01  0:00:01 --:--:--  133M


In [30]:
def unicode_to_greek(text):
    """
    Convert Unicode characters to their corresponding Greek letters.
    """
    greek_map = {
        u'\u03b1': u'α', u'\u03b2': u'β', u'\u03b3': u'γ', u'\u03b4': u'δ',
        u'\u03b5': u'ε', u'\u03b6': u'ζ', u'\u03b7': u'η', u'\u03b8': u'θ',
        u'\u03b9': u'ι', u'\u03ba': u'κ', u'\u03bb': u'λ', u'\u03bc': u'μ',
        u'\u03bd': u'ν', u'\u03be': u'ξ', u'\u03bf': u'ο', u'\u03c0': u'π',
        u'\u03c1': u'ρ', u'\u03c2': u'ς', u'\u03c3': u'σ', u'\u03c4': u'τ',
        u'\u03c5': u'υ', u'\u03c6': u'φ', u'\u03c7': u'χ', u'\u03c8': u'ψ',
        u'\u03c9': u'ω'
    }
    greek_text = ''
    for c in text:
        if c in greek_map:
            greek_text += greek_map[c]
        else:
            greek_text += c
    return greek_text

In [31]:
%pwd

'/content/ithaca/train/ithaca/train'

In [32]:
!python inference_example.py \
  --input_file=example_input.txt \
  --attribute_json=attribute.json \
  --restore_json=restore.json

python3: can't open file '/content/ithaca/train/ithaca/train/inference_example.py': [Errno 2] No such file or directory


In [33]:
class dataset_config:
  date_interval = 10
  date_max = 800
  date_min = -800

In [36]:
os.getcwd()

'/content/ithaca/train/ithaca/train'

In [35]:
import json
import numpy as np

def unicode_to_greek(text):
    """
    Convert Unicode characters to their corresponding Greek letters.
    """
    greek_map = {
        u'\u03b1': u'α', u'\u03b2': u'β', u'\u03b3': u'γ', u'\u03b4': u'δ',
        u'\u03b5': u'ε', u'\u03b6': u'ζ', u'\u03b7': u'η', u'\u03b8': u'θ',
        u'\u03b9': u'ι', u'\u03ba': u'κ', u'\u03bb': u'λ', u'\u03bc': u'μ',
        u'\u03bd': u'ν', u'\u03be': u'ξ', u'\u03bf': u'ο', u'\u03c0': u'π',
        u'\u03c1': u'ρ', u'\u03c2': u'ς', u'\u03c3': u'σ', u'\u03c4': u'τ',
        u'\u03c5': u'υ', u'\u03c6': u'φ', u'\u03c7': u'χ', u'\u03c8': u'ψ',
        u'\u03c9': u'ω'
    }
    greek_text = ''
    for c in text:
        if c in greek_map:
            greek_text += greek_map[c]
        else:
            greek_text += c
    return greek_text

# Specify the path to your JSON file here
json_path = 'restore.json'

# Load the JSON file
with open(json_path, 'r') as f:
    data = json.load(f)

# Extract the desired attribute from the JSON data
attribute = data['top_prediction']

# Convert the attribute to Greek letters
greek_attribute = unicode_to_greek(attribute)

with open('attribute.json', 'r') as f:
  attr = json.load(f)

data = attr["locations"]
max_location = max(data, key=lambda x: x["score"])
max_location_id = max_location["location_id"]
max_score = max_location["score"]
region_folder = '/content/ithaca/train/data/iphi-region-sub.txt'
year = attr["year_scores"]
id = year.index(max(year))

date_pred_y = np.array(attr["year_scores"])
date_pred_x = np.arange(
  dataset_config.date_min + dataset_config.date_interval / 2,
  dataset_config.date_max + dataset_config.date_interval / 2,
  dataset_config.date_interval)
date_pred_argmax = date_pred_y.argmax(
) * dataset_config.date_interval + dataset_config.date_min + dataset_config.date_interval // 2
date_pred_avg = np.dot(date_pred_y, date_pred_x)

with open(region_folder, "r") as file:
    for line in file:
        if str(max_location_id) in line:
            region_string = line.split('_')[0]
            break

# Print the result
print("INPUT")
with open('example_input.txt', 'r') as file:
    file_content = file.read()
    print(file_content)
print("RESULT : ")
print(greek_attribute)
print()
print("PREDICTION ACCURACY")
print(max_score)
print()
print("PREDICTED LOCATION :" )
print(region_string)
print()
print("PREDICTED TIMELINE : ")
if date_pred_avg < 0:
  date_pred_avg = abs(date_pred_avg)
  print(str(round(date_pred_avg)) + " BC")
else:
  print(str(date_pred_avg) + " AD")

FileNotFoundError: ignored

In [ ]:
greek_alphabet = {
    'Α': 'α', 'Β': 'β', 'Γ': 'γ', 'Δ': 'δ', 'Ε': 'ε',
    'Ζ': 'ζ', 'Η': 'η', 'Θ': 'θ', 'Ι': 'ι', 'Κ': 'κ',
    'Λ': 'λ', 'Μ': 'μ', 'Ν': 'ν', 'Ξ': 'ξ', 'Ο': 'ο',
    'Π': 'π', 'Ρ': 'ρ', 'Σ': 'σ', 'Τ': 'τ', 'Υ': 'υ',
    'Φ': 'φ', 'Χ': 'χ', 'Ψ': 'ψ', 'Ω': 'ω', 'Ϙ': 'ϙ',
    'Ϛ': 'ϛ'
}

def remove_erroneous_characters(string):
    cleaned_string = ''.join([char for char in string if char.isalpha() or char.isspace()])
    return cleaned_string

string = 'Μουσάων θεράποντα καὶ Ἡρακλῆος ἑταῖρον'
def convert_to_new_string(string):
    new_string = ''
    for char in string:
        if char in greek_alphabet:
            new_char = greek_alphabet[char]
            new_string += new_char
        else:
            new_string += char
    return new_string

print(convert_to_new_string(string))

In [ ]:
def read_file(file_name):
    lines = []
    with open(file_name, 'r') as file:
        for line in file:
            cleaned_input_string = remove_erroneous_characters(line)
            att = convert_to_new_string(cleaned_input_string)
            lines.append(att.strip())
    return lines

# Example usage
file_name = '/content/ithaca/greek text for testing.txt'  # Replace with your file name or path
string_array = read_file(file_name)
print(string_array)


In [ ]:
import random

def replace_with_question_marks(string_array):
    updated_array = []
    # i = 0
    for sentence in string_array:
        # if i > 2:
        #   break
        words = sentence.split()
        if words:
            random_word = random.choice(words)
            if len(random_word) > 3:
                replace_chars = min(3, len(random_word) - 1)
                question_marks = '?' * replace_chars + random_word[replace_chars:]
                updated_array.append([sentence.replace(random_word, question_marks), random_word[:3]])
            else:
                question_marks = '?' * len(random_word)
                updated_array.append([sentence.replace(random_word, question_marks), random_word])
        # i += 1
    return updated_array

updated_array = replace_with_question_marks(string_array)
for row in updated_array:
    print(row)



In [ ]:
%cd ithaca

In [ ]:
list_of_accuracies = [0.6625]

In [ ]:
import json
i = 0
list_of_accuracies = []

total = len(updated_array)
for line in updated_array:
  # print("PRINTING LINE NUMBER : ", i)
  # if i > 1:
  #   break
  with open("ex.txt", "w") as f:
    f.write(line[0])
  !python inference_example.py \
  --input_file=ex.txt \
  --attribute_json=attribute.json \
  --restore_json=restore.json
  json_path = 'restore.json'
  # Load the JSON file
  with open(json_path, 'r') as f:
      data = json.load(f)
      attribute = data['top_prediction']
  print()
  print("ACTUAL STRING : " +string_array[i])
  print("INPUT STRING : " + line[0])
  print("PREDICTED STRING : " + attribute)
  # print()
  count = 0
  # print(len(attribute))
  # print(len(string_array[i]))
  # print(len(line[0]))
  st = 0
  for j in range(len(line[0])):
    if line[0][j] == '?':
      st = j
      break
  k = 0
  for j in range(st, st + len(line[1])):
    if attribute[j] == line[1][k]:
      count += 1
    k += 1
  accuracy = count / len(line[1])
  print("ACCURACY : ", accuracy)
  print()
  list_of_accuracies.append(accuracy)
  i += 1

print()
print('MAX ACCURACY : ', max(list_of_accuracies))
print('MIN ACCURACY : ', min(list_of_accuracies))

In [ ]:
# Calculation of Overall Accuracy
sum=0
for i in range(len(list_of_accuracies)):
  sum+=list_of_accuracies[i]
acc=(sum/len(list_of_accuracies))*100

print("ACCURACY OF THE MODEL = ", round(acc,2))
